# Setup

In [1]:
import tensorflow as tf
from tensorflow.keras.models import model_from_json
import json
import os
from functools import reduce
import numpy as np
from copy import deepcopy

# Reading Data

In [2]:
# All the models from the Models folder will be stored in a dictionary
models_json_str = dict()
for filepath in os.listdir('Models'):
    with open(os.path.join('Models',filepath),'r') as model_file:
        filename, ext = os.path.splitext(filepath)
        if ext == '.json':
            models_json_str[filename] = model_file.read()

# Setting global variables

In [3]:
# Special Layers
MPOOL_LAYERS = ['MaxPooling1D','MaxPooling2D', 'MaxPooling3D']
APOOL_LAYERS = ['AveragePooling1D', 'AveragePooling2D', 'AveragePooling3D']
POOL_LAYERS = MPOOL_LAYERS + APOOL_LAYERS
GMPOOL_LAYERS = ['GloabalMaxPooling1D','GloabalMaxPooling2D', 'GloabalMaxPooling3D']
GAPOOL_LAYERS = ['GlobalAveragePooling1D', 'GlobalAveragePooling2D', 'GlobalAveragePooling3D']
GPOOL_LAYERS = GMPOOL_LAYERS + GAPOOL_LAYERS

# Extracting Data from the JSON file

In [9]:
class Model_Extractor:
    MPOOL_LAYERS = ['MaxPooling1D','MaxPooling2D', 'MaxPooling3D']
    APOOL_LAYERS = ['AveragePooling1D', 'AveragePooling2D', 'AveragePooling3D']
    POOL_LAYERS = MPOOL_LAYERS + APOOL_LAYERS
    GMPOOL_LAYERS = ['GloabalMaxPooling1D','GloabalMaxPooling2D', 'GloabalMaxPooling3D']
    GAPOOL_LAYERS = ['GlobalAveragePooling1D', 'GlobalAveragePooling2D', 'GlobalAveragePooling3D']
    GPOOL_LAYERS = GMPOOL_LAYERS + GAPOOL_LAYERS

    def __init__(self, model_json_str):
        self.model = model_from_json(model_json_str)
        self.model_json = json.loads(model_json_str)
        self.config = self.model_json['config']
        # self.target_layer_models = dict()
        self.outputs = dict()

        self.make_target_layer_models()
        
        self.get_output()
        self.dump_output('output.json')
        
        
    def get_layers(self):
        self.outputs['layers_ls'] = []
        for layer in self.model_json['config']['layers']:
            self.outputs['layers_ls'].append({'class_name': layer['class_name'],'name': layer['config']['name']})
        
        self.outputs['layers_count'] = len(self.outputs['layers_ls'])
    
    def get_layer_shape(self):
        self.outputs['layers_ls'][0]['shape'] = self.config['layers'][0]['config']['batch_input_shape'][1:]
        for layer in self.outputs['layers_ls'][1:]:
            layer['shape'] = [*self.model.get_layer(layer['name']).output_shape[1:]]

    def get_neurons(self):
        total_neuron_count = 0
        for layer in self.outputs['layers_ls']:
            layer['Neuron_count'] = reduce(lambda x,y : x*y, layer['shape'])
            total_neuron_count += layer['Neuron_count']
        
        self.outputs['Total_neuron_count'] = total_neuron_count

    def make_model_of_layer(self,target_layer_json: json):
        input_shape = [*self.model.get_layer(target_layer_json['config']['name']).input_shape]
        layer_model_json = deepcopy(self.model_json)
        layer_model_json['config']['layers'][0]['config']['batch_input_shape'] = input_shape

        for layer in self.model_json['config']['layers'][1:]:
            if layer == target_layer_json:
                continue
            layer_model_json['config']['layers'].remove(layer)
        
        if "activation" in layer_model_json['config']['layers'][1]['config']:
            layer_model_json['config']['layers'][1]['config']['activation'] = 'relu'

        layer_model = model_from_json(json.dumps(layer_model_json))
        return layer_model
    
    def make_target_layer_models(self):
        target_layer_models = {}
        for layer in self.config['layers'][1:]:
            target_layer_models[layer['config']['name']] = self.make_model_of_layer(layer)
        self.target_layer_models = target_layer_models
    
    def get_layer_connections(self,target_layer_json: json,layer_i: int):
        # layer_model = self.make_model_of_layer(target_layer_json)
        layer_model = self.target_layer_models[target_layer_json['config']['name']]
        input_shape = layer_model.layers[0].input_shape
        if layer_model.get_weights():
            weights_shape, biases_shape = layer_model.get_weights()
            weights_shape, biases_shape = weights_shape.shape, biases_shape.shape
            weights = np.zeros(weights_shape) + 1
            biases = np.zeros(biases_shape)
            layer_model.set_weights([weights,biases])
            extractor = tf.keras.Model(inputs=layer_model.inputs,outputs=layer_model.layers[0].output)

            features = extractor(np.zeros((1, *input_shape[1:]))+1)
            return features[0]
        if target_layer_json['class_name'] in POOL_LAYERS:
            pool_size = np.prod(target_layer_json['config']['pool_size'])
            # print(pool_size)
            no_of_neurons = self.outputs["layers_ls"][layer_i]["Neuron_count"]
            # print(no_of_neurons*pool_size)
            return no_of_neurons*pool_size
        return None

    def get_connections(self):
        # self.outputs['layers_ls'][0]['Layer connections'] = None
        self.outputs['layers_ls'][0]['Number of connections'] = None
        total_connections = 0
        for layer_i, (layer, out_layer) in enumerate(zip(self.config['layers'][1:],self.outputs['layers_ls'][1:]),start=1):
            # out_layer['Layer connections'] = self.get_layer_connections(layer)
            out_layer['Number of connections'] = np.sum(self.get_layer_connections(layer,layer_i))
            if out_layer['Number of connections']:
                out_layer['Number of connections'] = int(out_layer['Number of connections'])
                total_connections += out_layer['Number of connections']
        self.outputs['Total number of connections'] = total_connections
    
    def get_layer_multiplications(self, target_layer_json: json, layer_i: int):
        layer_model = self.target_layer_models[target_layer_json['config']['name']]
        input_shape = layer_model.layers[0].input_shape
        if layer_model.get_weights():
            weights_shape, biases_shape = layer_model.get_weights()
            weights_shape, biases_shape = weights_shape.shape, biases_shape.shape
            weights = np.zeros(weights_shape) + 1
            biases = np.zeros(biases_shape)
            layer_model.set_weights([weights,biases])
            extractor = tf.keras.Model(inputs=layer_model.inputs,outputs=layer_model.layers[0].output)

            features = extractor(np.zeros((1, *input_shape[1:]))+1)
            return int(np.sum(features[0]))
        if target_layer_json['class_name'] in APOOL_LAYERS:
            return 1
        return 0

    def get_multiplications(self):
        self.outputs['layers_ls'][0]['Number of multiplications'] = 0
        total_multiplications = 0
        total_divisions = 0
        for layer_i, (layer, out_layer) in enumerate(zip(self.config['layers'][1:],self.outputs['layers_ls'][1:]),start=1):
            # out_layer['Layer connections'] = self.get_layer_connections(layer)
            if layer['class_name'] in APOOL_LAYERS + GAPOOL_LAYERS:
                # Returns number of divisions in AveragePool layers
                out_layer['Number of divisions'] = self.get_layer_multiplications(layer,layer_i)
                total_divisions += out_layer['Number of divisions']

            else:
                out_layer['Number of multiplications'] = self.get_layer_multiplications(layer,layer_i)
                total_multiplications += out_layer['Number of multiplications']
        self.outputs['Total number of multiplications'] = total_multiplications
    
    def get_layer_additions(self,target_layer_json: json, layer_i):
        layer_model = self.target_layer_models[target_layer_json['config']['name']]
        input_shape = layer_model.layers[0].input_shape
        if layer_model.get_weights():
            weights_shape, biases_shape = layer_model.get_weights()
            weights_shape, biases_shape = weights_shape.shape, biases_shape.shape
            weights = np.zeros(weights_shape)
            biases = np.zeros(biases_shape) + 1
            layer_model.set_weights([weights,biases])
            extractor = tf.keras.Model(inputs=layer_model.inputs,outputs=layer_model.layers[0].output)

            features = extractor(np.zeros((1, *input_shape[1:])))
            return int(np.sum(features[0]))
        if target_layer_json['class_name'] in POOL_LAYERS:
            # Returns the number of additions for AveragePool layers and number of comparisions in MaxPool layers
            pool_size = np.prod(target_layer_json['config']['pool_size'])
            return int(pool_size-1)
        # if target_layer_json['class_name'] in MPOOL_LAYERS:

        return 0
        
    def get_additions(self):
        self.outputs['layers_ls'][0]['Number of additions'] = 0
        total_additions = 0
        total_comparisions = 0
        # for layer, out_layer in zip(self.config['layers'][1:],self.outputs['layers_ls'][1:]):
        for layer_i, (layer, out_layer) in enumerate(zip(self.config['layers'][1:],self.outputs['layers_ls'][1:]),start=1):
            # out_layer['Layer connections'] = self.get_layer_connections(layer)
            if layer['class_name'] in MPOOL_LAYERS + GMPOOL_LAYERS:
                # Records number of comparisions for MaxPool layers
                out_layer['Number of comparisions'] = self.get_layer_additions(layer,layer_i)
                total_comparisions += out_layer['Number of comparisions']
            else:
                # Records number of additions for AveragePool layers
                out_layer['Number of additions'] = self.get_layer_additions(layer, layer_i)
                total_additions += out_layer['Number of additions']
        self.outputs['Total number of additions'] = total_additions
        self.outputs['Total number of comparisions'] = total_comparisions

    def get_output(self):
        self.get_layers()
        # print(self)
        self.get_layer_shape()
        self.get_neurons()
        self.get_connections()
        self.get_multiplications()
        self.get_additions()

    def dump_output(self,filename):
        # Saves the output in a given file
        with open(filename,'w') as output_file:
            print(self.outputs)
            json.dump((self.outputs),output_file,indent=4, separators=(',',' : '))
    
def main():
    MNIST_convnet_extractor = Model_Extractor(models_json_str['Simple_MNIST_convnet'])
    # print(MNIST_convnet_extractor.outputs)
    print(MNIST_convnet_extractor.model.layers[0].dtype)

        # print((MNIST_convnet_extractor.outputs))
        # MNIST_convnet_extractor.outputs

if __name__ == "__main__":
    main()

{'layers_ls': [{'class_name': 'InputLayer', 'name': 'input_1', 'shape': [28, 28, 1], 'Neuron_count': 784, 'Number of connections': None, 'Number of multiplications': 0, 'Number of additions': 0}, {'class_name': 'Conv2D', 'name': 'conv2d', 'shape': [26, 26, 32], 'Neuron_count': 21632, 'Number of connections': 194688, 'Number of multiplications': 194688, 'Number of additions': 21632}, {'class_name': 'MaxPooling2D', 'name': 'max_pooling2d', 'shape': [13, 13, 32], 'Neuron_count': 5408, 'Number of connections': 21632, 'Number of multiplications': 0, 'Number of comparisions': 3}, {'class_name': 'Conv2D', 'name': 'conv2d_1', 'shape': [11, 11, 64], 'Neuron_count': 7744, 'Number of connections': 2230272, 'Number of multiplications': 2230272, 'Number of additions': 7744}, {'class_name': 'MaxPooling2D', 'name': 'max_pooling2d_1', 'shape': [5, 5, 64], 'Neuron_count': 1600, 'Number of connections': 6400, 'Number of multiplications': 0, 'Number of comparisions': 3}, {'class_name': 'Flatten', 'name':

# Rough

In [17]:
# json.dumps((MNIST_convnet_extract.outputs),indent=4, separators=('',' : '))
MNIST_convnet_extract.outputs
# with open('output.json', 'w') as 

{'layers_ls': [{'class_name': 'InputLayer',
   'name': 'input_1',
   'shape': [28, 28, 1],
   'Neuron_count': 784,
   'Number of connections': None,
   'Number of multiplications': 0,
   'Number of additions': 0},
  {'class_name': 'Conv2D',
   'name': 'conv2d',
   'shape': [26, 26, 32],
   'Neuron_count': 21632,
   'Number of connections': 194688.0,
   'Number of multiplications': 194688.0,
   'Number of additions': 21632.0},
  {'class_name': 'MaxPooling2D',
   'name': 'max_pooling2d',
   'shape': [13, 13, 32],
   'Neuron_count': 5408,
   'Number of connections': None,
   'Number of multiplications': 0,
   'Number of additions': 0},
  {'class_name': 'Conv2D',
   'name': 'conv2d_1',
   'shape': [11, 11, 64],
   'Neuron_count': 7744,
   'Number of connections': 2230272.0,
   'Number of multiplications': 2230272.0,
   'Number of additions': 7744.0},
  {'class_name': 'MaxPooling2D',
   'name': 'max_pooling2d_1',
   'shape': [5, 5, 64],
   'Neuron_count': 1600,
   'Number of connections': N

In [3]:
A = [1,2,3,4,5]
B = [6,7,8,9,10]
for i, (a,b) in enumerate(zip(A,B)):
    print(i,a,b)


0 1 6
1 2 7
2 3 8
3 4 9
4 5 10


In [8]:
MNIST_convnet_extract.get_layer_connections(MNIST_convnet_extract.config['layers'][-1])

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], dtype=float32)>

In [9]:
MNIST_convnet_extract.config['layers'][-1]

{'class_name': 'Dense',
 'config': {'name': 'dense',
  'trainable': True,
  'dtype': 'float32',
  'units': 10,
  'activation': 'softmax',
  'use_bias': True,
  'kernel_initializer': {'class_name': 'GlorotUniform',
   'config': {'seed': None}},
  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
  'kernel_regularizer': None,
  'bias_regularizer': None,
  'activity_regularizer': None,
  'kernel_constraint': None,
  'bias_constraint': None}}

In [6]:
MNIST_convnet_extract.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [7]:
MNIST_convnet_extract.model.layers[-1].get_weights()[0].shape

(1600, 10)

In [86]:
MNIST_convnet_extract.model.layers[0].output

<KerasTensor: shape=(None, 26, 26, 32) dtype=float32 (created by layer 'conv2d')>

In [81]:
MNIST_convnet_extract.model.layers[0].output_shape

(None, 26, 26, 32)

In [37]:
summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [62]:
tf.keras.utils.plot_model(MNIST_convnet_extract.model)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [83]:
import tensorflow.keras.layers as klayers


In [ ]:
from keras import backend as K
# x_input = np.zeros((28,28,1)) + 1
input_1 = MNIST_convnet_extract.model.input
output_1 = MNIST_convnet_extract.model.layers[0].output
fun = K.function([input_1,K.learning_phase()],output_1)

test = np.zeros((1,28,28,1)) + 1
layer_output = fun([test,1.])

In [89]:
MNIST_convnet_extract.model.input

<KerasTensor: shape=(None, 28, 28, 1) dtype=float32 (created by layer 'input_1')>

In [97]:
array = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(array)
array1 = array[np.newaxis]
print(array1)
print(np.array([array]))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]
[[[1 2 3]
  [4 5 6]
  [7 8 9]]]


In [120]:
model = MNIST_convnet_extract.model
# model.set_weights()
extractor = tf.keras.Model(inputs=model.inputs,outputs=[layer.output for layer in model.layers])

features = extractor(np.zeros((1,28,28,1))+1)

In [121]:
features

[<tf.Tensor: shape=(1, 26, 26, 32), dtype=float32, numpy=
 array([[[[9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          ...,
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.]],
 
         [[9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          ...,
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.]],
 
         [[9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          ...,
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.]],
 
         ...,
 
         [[9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          [9., 9., 9., ..., 9., 9., 9.],
          .

In [114]:
model.get_weights()[4].shape,model.get_weights()[5].shape

((1600, 10), (10,))

In [115]:
weights = [np.zeros((3,3,1,32))+1,np.zeros((32,)),np.zeros((3,3,32,64))+1,np.zeros((64,)),np.zeros((1600,10))+1,np.zeros((10,))]

In [116]:
model.set_weights(weights)

In [122]:
model.get_weights()[6]

IndexError: list index out of range

In [123]:
model.layers[0].input_shape

(None, 28, 28, 1)

In [1]:
1+None

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'